#### 排序不等式
设有两数列$a_1, a_2, ..., a_n$和$b_1, b_2, ..., b_n$，满足$a_1 \leq a_2 \leq ... \leq a_n$, $b_1 \leq b_2 \leq ... \leq b_n$，$c_1, c_2, ..., c_n$是$b_1, b_2, ..., b_n$的乱序排列，
则有$\sum\limits_{i=1}^{n}a_ib_i  \geq \sum\limits_{i=1}^{n}a_ic_i \geq \sum\limits_{i=1}^{n}a_ib_{n+1-i}$
例：
若$a_1 \leq a_2$、$b_1 \leq b_2$，
则$a_1b_1 \leq a_2b_2$

**例 道路的最大重要性 T2285**

In [2]:
def maximumImportance(n: int, roads: list[list[int]]) -> int:
    degree = [0 for _ in range(n)]
    for i, j in roads:
        degree[i] += 1
        degree[j] += 1
    res = 0
    for i, j in enumerate(sorted(degree), 1):
        res += i * j
    return res

print(maximumImportance(5, [[0,1],[1,2],[2,3],[0,2],[1,3],[2,4]]))

43


#### 线段树

In [ ]:
class Tree:
    def __init__(self, n, m):
        self.sum = [0] * (4 * n)

    # 给 idx 下标增加 val
    # self.add(1, 1, n, idx, val)
    def add(self, o: int, l: int, r: int, idx: int, val: int):
        if l == r:
            self.sum[l] += val
            return
        mid = (l + r) // 2
        if idx <= mid:
            self.add(o * 2, l, mid, idx, val)
        else:
            self.add(o * 2 + 1, mid + 1, r, idx, val)
        self.sum[o] = self.sum[o * 2] + self.sum[o * 2 + 1]

    # 返回 [L, R] 范围内的元素和
    # self.query_sum(1, 1, n, L, R)
    def query_sum(self, o: int, l: int, r: int, L: int, R: int):
        if L <= l and r <= R:
            return self.sum[o]
        sum = 0
        mid = (l + r) // 2
        if L <= mid:
            sum += self.query_sum(o * 2, l, mid, L, R)
        if R > mid:
            sum += self.query_sum(o * 2 + 1, mid + 1, r, L, R)
        return sum
